# Text Analysis through Natural Language Processing, Understanding and Classification

In [58]:
#Importing NLP packages
from wordcloud import WordCloud, STOPWORDS
from yellowbrick.text import FreqDistVisualizer
from sklearn.feature_extraction import stop_words
from textblob import TextBlob, Word
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import re
import pandas as pd # to dataframes
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

In [59]:
EDA=pd.read_excel('../CannaConnect/Dataset/EDA.xlsx')
EDA.head()

C:\Users\louis\Anaconda2\envs\py3.6\lib\site-packages\xlrd\xlsx.py:266: PendingDeprecationWarning:

This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.

C:\Users\louis\Anaconda2\envs\py3.6\lib\site-packages\xlrd\xlsx.py:312: PendingDeprecationWarning:

This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.



,Unnamed: 0,Strain,Type,Rating,Effects,Flavor,Description,Effect_1,Effect_2,Effect_3,Effect_4,Effect_5,Flavor_1,Flavor_2,Flavor_3,Flavor_4
0,0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,Creative,Energetic,Tingly,Euphoric,Relaxed,Earthy,Sweet,Citrus,NaN
1,1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,Relaxed,Aroused,Creative,Happy,Energetic,Flowery,Violet,Diesel,NaN
2,2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,Uplifted,Happy,Relaxed,Energetic,Creative,Spicy/Herbal,Sage,Woody,NaN
3,3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,Tingly,Creative,Hungry,Relaxed,Uplifted,Apricot,Citrus,Grapefruit,NaN
4,4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...",Happy,Relaxed,Euphoric,Uplifted,Talkative,Citrus,Earthy,Orange,NaN


In [60]:
#Preprocessing the text in the Description, print the head of the description text
stopwords = set(STOPWORDS)
newStopWords = ['strain','effect', 'genetic', 'effects','flavor','dominant','known','cross'] #from the list above, we are keeping words relevant to the analysis
stopwords.update(newStopWords)
df_des=EDA.Description.astype(str)
df_lower_case_des=df_des.apply(lambda x: " ".join(x.lower() for x in x.split())) #lower case
df_special_characters_out_des=df_lower_case_des.map(lambda x: re.sub(r'\W+', ' ',x)) #remove special character
df_stop_words_out_des=df_special_characters_out_des.apply(lambda x: " ".join(x for x in x.split() if x not in stopwords)) #remove stopwords
df_lemma_des=df_stop_words_out_des.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))  #lemmatize
print(df_lemma_des.head())

0    100 og 50 50 hybrid pack strong punch name sup...
1    98 aloha white widow especially potent cut whi...
2    1024 sativa hybrid bred spain medical seed co ...
3    13 dawgs hybrid g13 chemdawg genetics bred can...
4    kosher tangie 24k gold 60 indica hybrid combin...
Name: Description, dtype: object


In [61]:
df_digits = df_lemma_des.apply(lambda x: len([x for x in x.split() if x.isdigit()]))
print(df_digits.head())
print(df_digits.describe())

0    4
1    2
2    2
3    4
4    3
Name: Description, dtype: int64
count    2277.000000
mean        1.062363
std         1.390266
min         0.000000
25%         0.000000
50%         0.000000
75%         2.000000
max         8.000000
Name: Description, dtype: float64


In [62]:
df_numeric_out = df_lemma_des.str.replace('\\d+', '')
print(df_numeric_out.head())

0     og   hybrid pack strong punch name supposedly...
1     aloha white widow especially potent cut white...
2     sativa hybrid bred spain medical seed co bree...
3     dawgs hybrid g chemdawg genetics bred canadia...
4    kosher tangie k gold  indica hybrid combine le...
Name: Description, dtype: object


In [63]:
#Step 1: Identification of Common words
#Step 2: I am not going to remove common words at this point
common_words = pd.Series(' '.join(df_stop_words_out_des).split()).value_counts()[:10]
print(common_words)

s           1377
indica      1366
hybrid      1252
sativa      1060
og          1051
kush        1037
aroma        870
sweet        830
body         735
genetics     682
dtype: int64


In [64]:
#Step 1: Identification of Rare words
#Step 2: I am not going to rare words at this point
rare_words = pd.Series(' '.join(df_stop_words_out_des).split()).value_counts()[-10:]
print(rare_words)

densely        1
derivatives    1
skirt          1
werner         1
dripping       1
impending      1
blew           1
denseness      1
afkansastan    1
joke           1
dtype: int64


In [65]:
df_description_clean=df_numeric_out
df_description_clean.to_excel('../CannaConnect/Dataset/description_clean.xlsx')

In [66]:
#Visualization of the Frequent words
all_words = df_description_clean.str.split(expand=True).unstack().value_counts()
data = [go.Bar(
            x = all_words.index.values[1:50],
            y = all_words.values[1:50],
            marker= dict(colorscale='Jet',
                         color = all_words.values[2:100]
                        ),
            text='Word counts'
    )]

layout = go.Layout(
    title='Top 50 Word frequencies in the Dataset'
)

fig = go.Figure(data=data, layout=layout)

plot(fig, filename='basic-bar.html')

'basic-bar.html'